# Build a variational autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

The decoder part of the variational encoder can then be used to generate new reg numbers.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
from keras import optimizers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegDvlaVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegDvlaVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 150
actual number of validation samples = 1000
actual number of test samples = 1000


### Variational autoencoder

Design the architecture

In [3]:
latent_dim = 2

### Define the encoder
inputs = keras.Input(shape=(vec_length,))
h = inputs
h = layers.Dense(vec_length, activation='relu')(h)
h = layers.Dense(50, activation='relu')(h)
h = layers.Dense(25, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = latent_inputs
x = layers.Dense(25, activation='relu')(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dense(vec_length, activation='relu')(x)
x = layers.Dense(vec_length, activation='sigmoid')(x)
outputs = x
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= vec_length
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

Do some training

In [4]:
opt = optimizers.Adamax(learning_rate=0.004)
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

opt = optimizers.Adamax(learning_rate=0.001)
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

Epoch 1/200
1250/1250 [==============================] - 1s 1ms/step - loss: 28.3316 - val_loss: 27.1743
Epoch 2/200
1250/1250 [==============================] - 1s 926us/step - loss: 26.6140 - val_loss: 26.2119
Epoch 3/200
1250/1250 [==============================] - 1s 897us/step - loss: 25.5340 - val_loss: 24.9046
Epoch 4/200
1250/1250 [==============================] - 1s 927us/step - loss: 24.3768 - val_loss: 24.0247
Epoch 5/200
1250/1250 [==============================] - 1s 905us/step - loss: 23.7248 - val_loss: 23.5565
Epoch 6/200
1250/1250 [==============================] - 1s 913us/step - loss: 23.2875 - val_loss: 23.2307
Epoch 7/200
1250/1250 [==============================] - 1s 907us/step - loss: 23.0068 - val_loss: 22.9037
Epoch 8/200
1250/1250 [==============================] - 1s 921us/step - loss: 22.7920 - val_loss: 22.8846
Epoch 9/200
1250/1250 [==============================] - 1s 921us/step - loss: 22.6069 - val_loss: 22.5231
Epoch 10/200
1250/1250 [===============

Epoch 77/200
1250/1250 [==============================] - 1s 902us/step - loss: 21.5025 - val_loss: 22.0828
Epoch 78/200
1250/1250 [==============================] - 1s 905us/step - loss: 21.4919 - val_loss: 21.9913
Epoch 79/200
1250/1250 [==============================] - 1s 912us/step - loss: 21.4996 - val_loss: 21.9662
Epoch 80/200
1250/1250 [==============================] - 1s 897us/step - loss: 21.4710 - val_loss: 21.9437
Epoch 81/200
1250/1250 [==============================] - 1s 897us/step - loss: 21.4818 - val_loss: 22.0373
Epoch 82/200
1250/1250 [==============================] - 1s 911us/step - loss: 21.4760 - val_loss: 21.9877
Epoch 83/200
1250/1250 [==============================] - 1s 925us/step - loss: 21.4582 - val_loss: 22.0501
Epoch 84/200
1250/1250 [==============================] - 1s 926us/step - loss: 21.4648 - val_loss: 22.0309
Epoch 85/200
1250/1250 [==============================] - 1s 908us/step - loss: 21.4651 - val_loss: 21.9960
Epoch 86/200
1250/1250 [====

1250/1250 [==============================] - 1s 878us/step - loss: 21.2368 - val_loss: 22.0139
Epoch 153/200
1250/1250 [==============================] - 1s 847us/step - loss: 21.2529 - val_loss: 21.9793
Epoch 154/200
1250/1250 [==============================] - 1s 859us/step - loss: 21.2238 - val_loss: 21.9964
Epoch 155/200
1250/1250 [==============================] - 1s 872us/step - loss: 21.2239 - val_loss: 22.0385
Epoch 156/200
1250/1250 [==============================] - 1s 866us/step - loss: 21.2321 - val_loss: 21.9976
Epoch 157/200
1250/1250 [==============================] - 1s 877us/step - loss: 21.2351 - val_loss: 21.9851
Epoch 158/200
1250/1250 [==============================] - 1s 844us/step - loss: 21.2377 - val_loss: 21.9697
Epoch 159/200
1250/1250 [==============================] - 1s 877us/step - loss: 21.2274 - val_loss: 21.9462
Epoch 160/200
1250/1250 [==============================] - 1s 835us/step - loss: 21.2348 - val_loss: 21.9957
Epoch 161/200
1250/1250 [========

1250/1250 [==============================] - 1s 836us/step - loss: 20.8807 - val_loss: 21.9407
Epoch 28/200
1250/1250 [==============================] - 1s 830us/step - loss: 20.8794 - val_loss: 21.9311
Epoch 29/200
1250/1250 [==============================] - 1s 826us/step - loss: 20.8756 - val_loss: 21.9314
Epoch 30/200
1250/1250 [==============================] - 1s 825us/step - loss: 20.8719 - val_loss: 21.9490
Epoch 31/200
1250/1250 [==============================] - 1s 828us/step - loss: 20.8756 - val_loss: 21.9611
Epoch 32/200
1250/1250 [==============================] - 1s 828us/step - loss: 20.8722 - val_loss: 21.9506
Epoch 33/200
1250/1250 [==============================] - 1s 832us/step - loss: 20.8818 - val_loss: 21.9555
Epoch 34/200
1250/1250 [==============================] - 1s 833us/step - loss: 20.8732 - val_loss: 22.0041
Epoch 35/200
1250/1250 [==============================] - 1s 827us/step - loss: 20.8662 - val_loss: 21.9545
Epoch 36/200
1250/1250 [=================

Epoch 103/200
1250/1250 [==============================] - 1s 832us/step - loss: 20.7749 - val_loss: 22.0270
Epoch 104/200
1250/1250 [==============================] - 1s 832us/step - loss: 20.7790 - val_loss: 22.0744
Epoch 105/200
1250/1250 [==============================] - 1s 828us/step - loss: 20.7841 - val_loss: 21.9923
Epoch 106/200
1250/1250 [==============================] - 1s 831us/step - loss: 20.7828 - val_loss: 22.0117
Epoch 107/200
1250/1250 [==============================] - 1s 833us/step - loss: 20.7652 - val_loss: 21.9980
Epoch 108/200
1250/1250 [==============================] - 1s 826us/step - loss: 20.7723 - val_loss: 22.0086
Epoch 109/200
1250/1250 [==============================] - 1s 828us/step - loss: 20.7647 - val_loss: 22.0620
Epoch 110/200
1250/1250 [==============================] - 1s 826us/step - loss: 20.7817 - val_loss: 22.0174
Epoch 111/200
1250/1250 [==============================] - 1s 825us/step - loss: 20.7708 - val_loss: 22.0552
Epoch 112/200
1250/

1250/1250 [==============================] - 1s 922us/step - loss: 20.6904 - val_loss: 22.0753
Epoch 179/200
1250/1250 [==============================] - 1s 886us/step - loss: 20.6828 - val_loss: 22.1028
Epoch 180/200
1250/1250 [==============================] - 1s 973us/step - loss: 20.6938 - val_loss: 22.1197
Epoch 181/200
1250/1250 [==============================] - 1s 833us/step - loss: 20.6837 - val_loss: 22.0988
Epoch 182/200
1250/1250 [==============================] - 1s 952us/step - loss: 20.6934 - val_loss: 22.0971
Epoch 183/200
1250/1250 [==============================] - 1s 905us/step - loss: 20.6744 - val_loss: 22.0816
Epoch 184/200
1250/1250 [==============================] - 1s 873us/step - loss: 20.6850 - val_loss: 22.0480
Epoch 185/200
1250/1250 [==============================] - 1s 865us/step - loss: 20.6931 - val_loss: 22.1248
Epoch 186/200
1250/1250 [==============================] - 1s 979us/step - loss: 20.6907 - val_loss: 22.1674
Epoch 187/200
1250/1250 [========

Evaluation

In [5]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
recovered = [v.recover(x) for x in decoded_regs]
print(recovered[:10])
acc = np.sum([x == y for x, y in zip(recovered, test_strs)]) / len(test_strs)
print('overall accuracy = ' + str(acc))
print('by digit accuracy:')
for i_digit in range(len(test_strs[0])):
    acc = np.sum([x[i_digit] == y[i_digit] for x, y in zip(recovered, test_strs)]) / len(test_strs)
    print('    ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['SD33FCY', 'HL83OYL', 'NG19EUA', 'SL41GVA', 'EK91QKJ', 'BL47GFA', 'HP17XIP', 'WU89INF', 'HP57ETR', 'RH77RAW']
overall accuracy = 0.0
by digit accuracy:
    0.091
    0.058
    0.992
    0.987
    0.06
    0.073
    0.06


### Use decoder as generator

By directly sampling in the latent space, and running through the decoder, one effectively has a data generator.

In [6]:
[v.recover(x) for x in decoder.predict(np.random.random((5, latent_dim)))]

['PD64QJD', 'DV04STD', 'VS39DAR', 'HW64IGK', 'BF64VIT']

One can do some sampling and look at some stats to see what kind of data comes out. For instance, the number of reg numbers with pair of integers below 50 and above 50

In [13]:
random_vectors = 10 * (np.random.random((1000, latent_dim)) - 0.5)
generated_regs = [v.recover(x) for x in decoder.predict(random_vectors)]
n_reg_below_50 = len([x for x in generated_regs if int(x[2:4]) < 50])
n_reg_above_50 = len([x for x in generated_regs if int(x[2:4]) >= 50])
print('reg numbers below 50 = ' + str(n_reg_below_50))
print('reg numbers above 50 = ' + str(n_reg_above_50))

reg numbers below 50 = 462
reg numbers above 50 = 538


### Avoid generating any training data

There is a chance that a random generation of data might produce one of the actual registration numbers used in training. However, any input registrations are represented as a probability distribution in the latent space; so one could require that any sample vector used in the latent space to be 'sufficiently far' down the tails of all the recorded distributions.

The distributions of the training data in the latent space are readily available:

In [34]:
# Get the distributions for each reg number in the data
all_strs = train_strs + val_strs + test_strs
all_vecs = np.array([v.vectorize(x) for x in all_strs])
means, log_sigmas, _ = encoder.predict(all_vecs)
distn_dict = {x: (y,z) for x, y, z in zip(all_strs, means, np.exp(log_sigmas))}  # dict<reg no: (mean, sigma)>
example_reg = list(distn_dict.keys())[0]
print('distribution for input data registration ' + str(example_reg))
print('    mu = ' + str(distn_dict[example_reg][0]))
print('    diagonal sigmas = ' + str(distn_dict[example_reg][1]))
# for a multivariate distribution ~ N(mean, Var) valued in the latent space, 
# mean is the first vector below, and Var is a diagonal matrix, the square root
# of the diagonal entries being given in the second vector.
# If the KL loss function pulls the distributions to be close to N(0,1)

distribution for input data registration YK66BIQ
    mu = [-0.16170382 -0.5909832 ]
    diagonal sigmas = [0.20191897 0.21653965]


### Generating subsets of data with certain characteristics

As a more general example of 'avoid generating any training data', consider the case of 'avoid generating data with certain characteristics'. Since the latent space has a joint probability density function over it, this can be answered in a statistical manner. Consider the distributions of all data, that represent numbers below 50 (recall all 7 digits are alpha numeric, with only the third and fourth digits being numeric):

In [15]:
distns_below_50 = {k: v for k, v in distn_dict.items() if int(k[2:4]) < 50}
print('number of distributions with number below 50 = ' + str(len(distns_below_50)))

number of distributions with number below 50 = 6103


and use these to find only random vectors that are 'sufficiently far away' from the means of those reg numbers with numbers below 50. So the 'characteristic' chosen, will be for the number to be above 50.

Employing a cheap-and-cheerful example of rejection sampling, and a cheap-and-cheerful example of statistical hypothesis testing, one can obtain the a set of random vectors that are 'sufficiently far' in the tails of the distributions belonging to those data vectors where the registration numbers are below 50.

In [19]:
means_mtr = np.array([mu for mu, sigma in distns_below_50.values()])
sigma_mtr = np.array([sigma for mu, sigma in distns_below_50.values()])
min_mahalanobis_dists = [np.min(np.linalg.norm((x - means_mtr) / sigma_mtr, axis=1))
                         for x in random_vectors]


In [26]:
mahalanobis_threshold = 3.5
filtered_random_vectors = [x for x, y in zip(random_vectors, np.array(min_mahalanobis_dists)/latent_dim) 
                           if y > mahalanobis_threshold]
print('number of filtered random vectors = ' + str(len(filtered_random_vectors)))

number of filtered random vectors = 40


In [35]:
filtered_generated_regs = [v.recover(x) for x in decoder.predict(np.array(filtered_random_vectors))]
filtered_n_reg_above_50 = len([x for x in filtered_generated_regs if int(x[2:4]) > 50])
print('number of filtered reg numbers with numeric part above 50 = ' + str(filtered_n_reg_above_50))

number of filtered reg numbers with numeric part above 50 = 37
